In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')

In [4]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [5]:
pd.read_sql(
    """
    SELECT * FROM boston
   
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [6]:
pd.read_sql(
    """
    SELECT COUNT(*) - COUNT(CRIM) as CRIM_nulls,  COUNT(*) - COUNT(ZN) as ZN_nulls, COUNT(*) - COUNT(INDUS) as INDUS_nulls, COUNT(*) - COUNT(NOX) as NOX_nulls 
    FROM boston
    
      
    
    """,
    con,
)

,CRIM_nulls,ZN_nulls,INDUS_nulls,NOX_nulls
0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [7]:
pd.read_sql(
    """
    SELECT COUNT(DISTINCT CRIM) as uniq_CRIM, COUNT(DISTINCT ZN) as uniq_ZN, COUNT(DISTINCT INDUS) as uniq_INDUS, COUNT(DISTINCT CHAS) as uniq_CHAS, COUNT(DISTINCT NOX) as uniq_NOX 
    FROM boston 
  
    """,
    con,
)

,uniq_CRIM,uniq_ZN,uniq_INDUS,uniq_CHAS,uniq_NOX
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [8]:

pd.read_sql(
    
    """

    WITH CRIM1 as (SELECT CRIM FROM boston
    ORDER BY CRIM
    LIMIT 1
    OFFSET (SELECT COUNT(*) FROM boston) / 2),
    
    ZN1 as (SELECT ZN FROM boston
    ORDER BY ZN
    LIMIT 1
    OFFSET (SELECT COUNT(*) FROM boston) / 2),
    
    INDUS1 as (SELECT INDUS FROM boston
    ORDER BY INDUS
    LIMIT 1
    OFFSET (SELECT COUNT(*) FROM boston) / 2),
    
    CHAS1 as (SELECT CHAS FROM boston
    ORDER BY CHAS
    LIMIT 1
    OFFSET (SELECT COUNT(*) FROM boston) / 2),
    
    NOX1 as (SELECT NOX FROM boston
    ORDER BY NOX
    LIMIT 1
    OFFSET (SELECT COUNT(*) FROM boston) / 2),
    
    MINALL as (SELECT min(CRIM) as mincrim, min(ZN) as minzn, min(INDUS) as minindus, min(CHAS) as minchas, min(NOX) as minnox FROM boston),
    
    COMBINE as (SELECT * from MINALL 
    join CRIM1
    join ZN1
    join INDUS1
    join CHAS1
    join NOX1)
    
    SELECT 
    iif(mincrim-CRIM=0, 'CRIM', ' ') ||
    iif(minzn-ZN=0, 'ZN', ' ') ||
    iif(minindus-INDUS=0, 'INDUS', ' ') ||
    iif(minchas-CHAS=0, 'CHAS', ' ') ||
    iif(minnox-NOX=0, 'NOX', ' ') as res
    from COMBINE
    
  
   
    
    """,
    con,
)

,res
0,ZN CHAS


In [ ]:
В колонках ZN и CHAS низкая вариативность значений, значительную долю занимают нулевые значения. 

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [9]:
pd.read_sql(
    """
WITH RICH25 as (SELECT RM
FROM boston
ORDER BY MEDV DESC
LIMIT 25),

POOR25 as (SELECT RM
FROM boston
ORDER BY MEDV
LIMIT 25),

RICH50 as (SELECT RM
FROM boston
ORDER BY MEDV DESC
LIMIT 50),

POOR50 as (SELECT RM
FROM boston
ORDER BY MEDV
LIMIT 50),

RICH100 as (SELECT RM
FROM boston
ORDER BY MEDV DESC
LIMIT 100),

POOR100 as (SELECT RM
FROM boston
ORDER BY MEDV
LIMIT 100),

RICH200 as (SELECT RM
FROM boston
ORDER BY MEDV DESC
LIMIT 200),

POOR200 as (SELECT RM
FROM boston
ORDER BY MEDV
LIMIT 200),

RICH300 as (SELECT RM
FROM boston
ORDER BY MEDV DESC
LIMIT 300),

POOR300 as (SELECT RM
FROM boston
ORDER BY MEDV
LIMIT 300),

GETRICH25 as (SELECT AVG(RM) as RICH25 FROM RICH25),
GETPOOR25 as (SELECT AVG(RM) as POOR25 FROM POOR25),

GETRICH50 as (SELECT AVG(RM) as RICH50 FROM RICH50),
GETPOOR50 as (SELECT AVG(RM) as POOR50 FROM POOR50),

GETRICH100 as (SELECT AVG(RM) as RICH100 FROM RICH100),
GETPOOR100 as (SELECT AVG(RM) as POOR100 FROM POOR100),

GETRICH200 as (SELECT AVG(RM) as RICH200 FROM RICH200),
GETPOOR200 as (SELECT AVG(RM) as POOR200 FROM POOR200),

GETRICH300 as (SELECT AVG(RM) as RICH300 FROM RICH300),
GETPOOR300 as (SELECT AVG(RM) as POOR300 FROM POOR300),

COMBINE as (SELECT * FROM GETRICH25
join GETPOOR25
join GETRICH50
join GETPOOR50
join GETRICH100
join GETPOOR100
join GETRICH200
join GETPOOR200
join GETRICH300
join GETPOOR300
)

SELECT RICH25, POOR25, RICH25-POOR25 as DIFF25, RICH50, POOR50, RICH50-POOR50 as DIFF50, RICH100, POOR100, RICH100-POOR100 as DIFF100, RICH200, POOR200, RICH200-POOR200 as DIFF200, RICH300, POOR300, RICH300-POOR300 as DIFF300
from combine


    """,
    con,
)

,RICH25,POOR25,DIFF25,RICH50,POOR50,DIFF50,RICH100,POOR100,DIFF100,RICH200,POOR200,DIFF200,RICH300,POOR300,DIFF300
0,7.63732,5.74784,1.88948,7.48706,5.75324,1.73382,7.19876,5.88712,1.31164,6.75868,5.911705,0.846975,6.539007,5.972227,0.56678


In [ ]:
Чем больше выборка, тем меньше отслеживается разница. Но в целом, чем больше комнат, тем дороже жилье.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех знаечений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [10]:
pd.read_sql(
    """
    


SELECT LSTAT, MEDV, RANK() OVER(ORDER BY LSTAT) rnk
FROM boston
WHERE MEDV = (SELECT Max(MEDV) from boston)
ORDER BY MEDV DESC


    """,
    con,
)

,LSTAT,MEDV,rnk
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,3
3,2.96,50.0,4
4,2.97,50.0,5
5,3.16,50.0,6
6,3.26,50.0,7
7,3.32,50.0,8
8,3.70,50.0,9
9,3.73,50.0,10


In [ ]:
В домах с самой дорогой стоимостью самый низкий процент населения с более низким статусом.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [11]:
pd.read_sql(
    """
    SELECT AVG(MEDV) FROM boston
    GROUP BY CHAS
   
    """,
    con,
)

,AVG(MEDV)
0,22.093843
1,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [12]:
pd.read_sql(
    """
    SELECT Result FROM ( WITH
    CHAST AS (
    SELECT DISTINCT CHAS,
    AVG(CRIM) OVER (PARTITION BY CHAS) AS ACRIM,
    AVG(ZN) OVER (PARTITION BY CHAS) AS AZN,
    AVG(INDUS) OVER (PARTITION BY CHAS) AS AINDUS,
    AVG(NOX) OVER (PARTITION BY CHAS) AS ANOX
    FROM boston
    WHERE CHAS = 1),
    CHASF AS (
    SELECT DISTINCT CHAS,
    AVG(CRIM) OVER (PARTITION BY CHAS) AS ACRIM,
    AVG(ZN) OVER (PARTITION BY CHAS) AS AZN,
    AVG(INDUS) OVER (PARTITION BY CHAS) AS AINDUS,
    AVG(NOX) OVER (PARTITION BY CHAS) AS ANOX
    FROM boston
    WHERE CHAS = 0)
    
    SELECT 'CRIM' AS 'Result', CHAST.ACRIM - CHASF.ACRIM AS 'Diff' FROM CHAST, CHASF
    UNION ALL
    SELECT 'ZN', CHAST.AZN - CHASF.AZN FROM CHAST, CHASF
    UNION ALL
    SELECT 'INDUS', CHAST.AINDUS - CHASF.AINDUS FROM CHAST, CHASF
    UNION ALL
    SELECT 'NOX', CHAST.ANOX - CHASF.ANOX FROM CHAST, CHASF
    )
    
    WHERE Diff > 0
    """,
    con,
)

,Result
0,INDUS
1,NOX


In [ ]:
У реки больше промышленных зданий и следующее за этим большее загрязнение воздуха

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [13]:
pd.read_sql(
    """
WITH data AS(    
    SELECT *,  ntile(10) over(order by indus) as pindus,
    ntile(10) over(order by nox) as pnox   
    FROM boston)
    

SELECT  t1.perc, nox_value, indus_value

FROM (select distinct pindus*10 as perc, max(indus) over(partition by pindus) indus_value
        from data ) t1 join
     (select distinct pnox*10 as perc, max(nox) over(partition by pnox) nox_value from data) t2

ON t1.perc = t2.perc
     


    """,
    con,
)

,perc,nox_value,indus_value
0,10,0.426,2.89
1,20,0.442,4.39
2,30,0.472,5.96
3,40,0.507,7.38
4,50,0.538,9.69
5,60,0.580,13.89
6,70,0.609,18.10
7,80,0.671,18.10
8,90,0.713,19.58
9,100,0.871,27.74


In [ ]:
Концентрация оксидов азота напрямую зависит от концентрации неторгового бизнеса